# Logging

Logging enables information about the current state and execution of a program to be outputted for debugging purposes. We already did use some limited form of logging utilizing `System.out` to write information to standard output. While this already is useful, this method has some drawbacks especifically in the context of larger, more complex programs:

1. There is no mechansim for conveniently deciding what information should be logged. For example, during development we want to be able to see all possible log output while in production we would like to limit log output to the bare essentials to reduce the overhead of logging.
2. All log output is printed to the console. This is often appropriate for debugging of a single-node application, but when building a distributed system or in production we may prefer writing log output to a file, possibly even rotating among a set of log files of fixed size, or send it to a server, or use multiple targets at once.

The purpose of logging frameworks is to allow this type of flexible logging. Java has a build-in logging framework in package `java.logging`.



In [ ]:
import java.logging.*;

